I need a small testing playground that won't crash on my laptop, so I'm creating one here for the 10000 frame centriole data.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
filename = Path('../test-data/acTub_COT_gain100_30ms/acTub_COT_gain100_30ms.csv')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

In [3]:
df.describe()

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background
count,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000
mean,2425.196663,35.420145,2182.946881,35.420145,5172.075346,6288.638717,1029.127987,744.422318
std,1595.345619,20.475942,1322.208205,20.475942,2777.221724,15586.487514,2896.254531,312.086668
min,0.000001,0.847267,0.000002,0.847267,0.000000,338.324000,50.035500,523.371000
25%,965.218000,20.430100,982.750000,20.430100,2904.000000,577.636000,222.657000,613.929000
50%,2403.570000,38.192700,2206.350000,38.192700,5165.000000,772.204000,304.708000,656.426000
75%,3892.590000,50.197000,3461.780000,50.197000,7563.000000,1596.560000,433.924000,730.572000
max,4900.000000,80.431400,4200.000000,80.431400,9999.000000,177421.000000,58298.300000,7147.530000


In [8]:
import importlib

In [68]:
maxSize = np.max([df['x'].max(),df['y'].max()])
pixelSize = 100 # nm

fig = plt.figure()
ax  = fig.add_subplot(111)
ax.hist2d(df['x'], df['y'], bins = maxSize / 100)
ax.invert_yaxis()
plt.show()

In [161]:
plt.close()

In [172]:
importlib.reload(ds)

<module 'DataSTORM.processors' from '/home/kmdouglass/src/DataSTORM/DataSTORM/processors.py'>

In [173]:
corr = ds.FiducialDriftCorrect(mergeRadius      = 90,
                               offTime          = 3,
                               minSegmentLength = 30,
                               searchRegions    = [{'xMin' : 1500,
                                                    'xMax' : 1700,
                                                    'yMin' : 2500,
                                                    'yMax' : 2700}])

In [174]:
corr.iSearch(df)

Space bar pressed!
Space bar pressed!
Space bar pressed!
Closed Figure!
Done!


/home/kmdouglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/matplotlib/backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [159]:
plt.close()

In [170]:
df2 = corr(df)

Frame 9996: 1 trajectories present
1 fiducial(s) detected.


In [171]:
df2.describe()

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background,x-drift,y-drift
count,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000
mean,2420.165809,35.420145,2208.835152,35.420145,5172.075346,6288.638717,1029.127987,744.422318,5.030854,-25.888271
std,1595.257596,20.475942,1322.741519,20.475942,2777.221724,15586.487514,2896.254531,312.086668,6.307410,21.354215
min,-15.199535,0.847267,-2.116667,0.847267,0.000000,338.324000,50.035500,523.371000,-14.685178,-65.347033
25%,960.872937,20.430100,1008.752163,20.430100,2904.000000,577.636000,222.657000,613.929000,4.129510,-47.686491
50%,2397.731463,38.192700,2231.733309,38.192700,5165.000000,772.204000,304.708000,656.426000,7.131784,-24.689144
75%,3886.908632,50.197000,3478.135585,50.197000,7563.000000,1596.560000,433.924000,730.572000,8.537687,-4.239633
max,4914.684737,80.431400,4265.347033,80.431400,9999.000000,177421.000000,58298.300000,7147.530000,15.201846,2.118678


In [175]:
corr.searchRegions

[{'xMax': None, 'xMin': None, 'yMax': None, 'yMin': None}]

In [96]:
corr.resetSearchRegions()
corr.searchRegions

[{'xMax': None, 'xMin': None, 'yMax': None, 'yMin': None}]